# Working with Raw Gaze Samples

The term "raw gaze" or "raw eye-tracking data" is used inconsistently in the literature and can refer to different levels of data, depending on context. Common usages include:

- **Raw eye-tracker files**: May contain samples, events, messages, and metadata mixed together.
- **Raw samples**: Gaze coordinates over time without filtering or event classification.
- **Vendor-labeled events**: Fixations or saccades produced by proprietary software.

In pymovements, raw samples refer to the lowest-level gaze data available after import, before any additional preprocessing steps, such as smoothing, velocity computation, or event detection. These raw samples form the foundation for all subsequent analyses. Later transformations, e.g. converting pixel coordinates to degrees of visual angle, computing velocities, or segmenting fixations and saccades, operate on these samples and depend on the assumptions and metadata established during
loading.

The table below shows a simplified example of raw gaze samples after import ({py:class}`~pymovements.Gaze`). Each row corresponds to one time-ordered gaze sample and is stored in the ``samples`` attribute.

In [ ]:
import warnings

import matplotlib.pyplot as plt

import pymovements as pm
from pymovements.gaze.experiment import Experiment

experiment = Experiment(
    screen_width_px=1680,
    screen_height_px=1050,
    screen_width_cm=47.5,
    screen_height_cm=30,
    distance_cm=65,
    origin='upper left',
    sampling_rate=1000.0,
)

with warnings.catch_warnings():
    warnings.filterwarnings(
        "ignore",
        message="Gaze contains samples but no components could be inferred.*",
        category=UserWarning,
    )
    gaze = pm.gaze.from_csv(
        "../examples/gaze-toy-10s.csv",
        experiment=experiment,
        time_column="timestamp",
    )

# drop columns stimuli_x and stimuli_y
gaze.samples = gaze.samples.drop(['stimuli_x', 'stimuli_y'])
gaze.samples.head(8)

Column dtypes after import:

- ``time`` (``i64``): timestamp of the sample, typically in milliseconds.
- ``x`` (``f64``): horizontal gaze position in pixel coordinates.
- ``y`` (``f64``): vertical gaze position in pixel coordinates.
- ``pupil_diameter`` (``f64``): estimate of pupil size in arbitrary, device-dependent units.

In [ ]:
gaze1 = pm.gaze.from_csv(
    "../examples/gaze-toy-10s.csv",
    experiment=experiment,
    time_column="timestamp",
    pixel_columns=['x', 'y']
)

# drop columns stimuli_x and stimuli_y
gaze1.samples = gaze1.samples.drop(['stimuli_x', 'stimuli_y'])
gaze1.samples.head(8)

## Inspecting Raw Samples with Time-Series Plots

Time-series plots are often the first step when working with newly loaded gaze data. They provide a direct view of the temporal structure of the signal and help assess data quality before any preprocessing or event detection is applied.

Using the {py:func}`~pymovements.plotting.traceplot` function, we can visualize raw gaze samples from a {py:class}`~pymovements.Gaze` object. The plot shows the continuous trajectory of gaze positions across the stimulus, allowing inspection of spatial gaze behavior over time. 

In [ ]:
pm.plotting.traceplot(gaze1)

plt.show()

The {py:func}`~pymovements.plotting.tsplot` function produces a time-series plot of gaze samples from a {py:class}`~pymovements.Gaze` object. The plot shows how each recorded signal changes over time, with one line per selected channel (e.g., horizontal and vertical gaze position).The x-axis represents time, as defined by the gaze sample timestamps.

In this example, we plot the `x` and `y` channels to examine the raw gaze signal before applying any event detection or preprocessing. 

In [ ]:
pm.plotting.tsplot(
    gaze,
    channels=['x', 'y'],
    share_y=False,
    line_color="darkblue",
    zero_centered_yaxis=False)

plt.show()

Time-series inspection can reveal common issues such as signal loss, noise, blinks, sampling irregularities, or calibration problems, which may strongly influence subsequent analysis steps.

See the {doc}`Plotting Gaze Data tutorial <../tutorials/plotting>` for an example of time-series visualization using ``traceplot``.

## Converting Units to Standardized Representations

Raw gaze samples form the basis of all subsequent analysis, but meaningful interpretation often requires transforming these samples into alternative representations. These transformations operate directly on raw samples and typically precede any event detection or higher-level segmentation.

### From Pixels to Degrees of Visual Angle

Pixel coordinates depend on screen resolution, viewing distance, and physical screen size. To compare gaze behaviour across setups or participants, it is often useful to convert pixels to degrees of visual angle (dva). This conversion requires knowledge of the experimental geometry and is handled explicitly in pymovements by the {py:func}`~pymovements.gaze.transforms.pix2deg` function.

### From Position to Velocity

Many eye-movement measures are derived not from position directly but from its temporal
derivatives. Velocity is computed from changes in gaze position over time and is central to event detection algorithms for saccades and fixations. In pymovements, velocity is computed explicitly from position data with the {py:func}`~pymovements.gaze.transforms.pos2vel` function, using the sampling rate stored in the eye tracker definition.